# Introduction

* Datasets:
    * https://huggingface.co/datasets/timdettmers/openassistant-guanaco/viewer/default/train?row=0
* Models:
    * https://huggingface.co/state-spaces/mamba-130m-hf

In [1]:
!pip install -U accelerate transformers trl datasets

In [2]:
import os
import torch

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

## Configuration

In [3]:
batch_size = 4
num_workers = os.cpu_count()
# max_steps = -1 for epoch-wise training.
# epochs = -1 for step-wise training.
# Both cannot be -1.
max_steps = -1
epochs = 3
bf16 = False
fp16 = True
gradient_accumulation_steps = 8
seq_length = 1024
logging_steps = 50
save_steps = 50
learning_rate = 0.0001
model_name = 'state-spaces/mamba-130m-hf'
out_dir = 'outputs/mamba_130m_oasst_guanaco_sft'
seed = 42

## Load Dataset 

In [4]:
dataset = load_dataset('timdettmers/openassistant-guanaco')

/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9846
    })
    test: Dataset({
        features: ['text'],
        num_rows: 518
    })
})


In [6]:
print(dataset['train']['text'][0])

### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading

## Model

In [7]:
if bf16:
    model = AutoModelForCausalLM.from_pretrained(model_name).to(dtype=torch.bfloat16)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name)

In [8]:
print(model)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

MambaForCausalLM(
  (backbone): MambaModel(
    (embeddings): Embedding(50280, 768)
    (layers): ModuleList(
      (0-23): 24 x MambaBlock(
        (norm): MambaRMSNorm()
        (mixer): MambaMixer(
          (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
          (act): SiLU()
          (in_proj): Linear(in_features=768, out_features=3072, bias=False)
          (x_proj): Linear(in_features=1536, out_features=80, bias=False)
          (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
          (out_proj): Linear(in_features=1536, out_features=768, bias=False)
        )
      )
    )
    (norm_f): MambaRMSNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50280, bias=False)
)
129,135,360 total parameters.
129,135,360 training parameters.


## Tokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code=True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Training

In [10]:
if max_steps == -1 and epochs > 0:
    training_args = TrainingArguments(
        output_dir=f"{out_dir}/logs",
        evaluation_strategy='epoch',
        weight_decay=0.01,
        load_best_model_at_end=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_strategy='steps',
        save_strategy='epoch',
        logging_steps=logging_steps,
        num_train_epochs=epochs,
        save_total_limit=2,
        bf16=bf16,
        fp16=fp16,
        report_to='tensorboard',
        dataloader_num_workers=num_workers,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        lr_scheduler_type='constant',
        seed=seed
    )

if max_steps > 0 and epochs == -1:
    training_args = TrainingArguments(
        output_dir=f"{out_dir}/logs",
        evaluation_strategy='steps',
        weight_decay=0.01,
        load_best_model_at_end=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_strategy='steps',
        save_strategy='steps',
        logging_steps=logging_steps,
        save_steps=save_steps,
        save_total_limit=2,
        bf16=bf16,
        fp16=fp16,
        report_to='tensorboard',
        max_steps=max_steps,
        dataloader_num_workers=num_workers,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        lr_scheduler_type='constant',
        seed=seed
    )

In [11]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    dataset_text_field='text',
    max_seq_length=seq_length,
    tokenizer=tokenizer,
    args=training_args,
    packing=True
)

/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [12]:
dataloader = trainer.get_train_dataloader()
for i, sample in enumerate(dataloader):
    print(tokenizer.decode(sample['input_ids'][0]))
    print('#'*50)
    if i == 5:
        break

 al gusto.
3. Coloca la mezcla en una bandeja para horno y agrega la rebanada de jamón encima.
4. Espolvorea el queso rallado sobre la mezcla y hornea a 180°C durante 10-15 minutos o hasta que el queso esté derretido y dorado.

¡Listo! Ya tienes una deliciosa tostada de jamón y queso con cebolla sin necesidad de utilizar aceite o gas. ¡Buen provecho!### Human: Pero si un horno usa gas!. Y si te refieres a un horno eléctrico, se descompuso la semana pasada. Necesito una receta en la que no se necesite cocción o calentamiento de algún alimento por favor.<|endoftext|>### Human: Hola, ¿cómo estás?### Assistant: Hola, muy bien, gracias por preguntar. ¿En qué puedo ayudarte hoy?<|endoftext|>### Human: I'm a 26 year old, 6foot tall male, I weight 130 pounds. I want to gain some weight, mostly muscle though. Can you give me a workout plan for a whole week and a meal plan that provides me with the calories needed?### Assistant: Sure! Here is a sample workout plan for a week, as well as a sample

In [13]:
history = trainer.train()

Epoch,Training Loss,Validation Loss
0,2.073600,2.076869
1,1.926900,2.057147
2,1.833800,2.059122


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [14]:
model.save_pretrained(f"{out_dir}/best_model")
tokenizer.save_pretrained(f"{out_dir}/best_model")

('outputs/mamba_130m_oasst_guanaco_sft/best_model/tokenizer_config.json',
 'outputs/mamba_130m_oasst_guanaco_sft/best_model/special_tokens_map.json',
 'outputs/mamba_130m_oasst_guanaco_sft/best_model/tokenizer.json')

## Inference

In [4]:
from transformers import (
    AutoModelForCausalLM, 
    logging, 
    pipeline,
    AutoTokenizer
)

import torch

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
model = AutoModelForCausalLM.from_pretrained('outputs/mamba_130m_oasst_guanaco_sft/best_model/')
tokenizer = AutoTokenizer.from_pretrained('outputs/mamba_130m_oasst_guanaco_sft/best_model/')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
print(tokenizer.eos_token)

<|endoftext|>


In [8]:
pipe = pipeline(
    task='text-generation', 
    model=model, 
    tokenizer=tokenizer, 
    max_length=1024, # Prompt + new tokens to generate.
    device_map=device,
    # eos_token_id=tokenizer.eos_token_id
)

In [9]:
logging.set_verbosity(logging.CRITICAL)

In [10]:
template = '### Human: {}### Assistant:'

In [15]:
prompt = "Give ten tips for staying healthy."
prompt = template.format(prompt)

outputs = pipe(
    prompt,
    repetition_penalty=1.1,
)
print(outputs[0]['generated_text'])

### Human: Give ten tips for staying healthy.### Assistant: Here are 10 tips to stay healthy that can help you live a healthier life and achieve your goals in the shortest amount of time possible, without getting too stressed or overwhelmed with workouts!
1. Eat a balanced diet every day - This includes whole grains like whole-grain breads (such as brown rice), fruits such as bananas, apples, and berries; vegetables including carrots, celery, cabbage, and onions; lean protein sources from fish, poultry, and eggs; and plenty of nuts/seeds throughout the week.
2. Get enough sleep - The night before any activity is important because it helps regulate blood sugar levels during rest periods so you don't get dehydrated overnight when you're not feeling well.
3. Stay hydrated - Drink plenty of water at least twice per day while still active but less than three hours after waking up each morning.
4. Exercise regularly - Regular physical activities include walking, cycling, swimming, running, e

In [17]:
prompt = "How to be good at mathematics?"
prompt = template.format(prompt)

outputs = pipe(
    prompt,
    # do_sample=True,
    # temperature=0.7,
    # top_k=50,
    # top_p=0.95,
    repetition_penalty=1.1,
)
print(outputs[0]['generated_text'])

### Human: How to be good at mathematics?### Assistant: Mathematics is a highly technical and challenging subject that requires the ability for problem-solving, reasoning skills, and critical thinking. It's important not only in school but also during your career as an engineer or manager who must work with complex problems on multiple levels of complexity from design through implementation.
Here are some tips you can follow when working effectively within this field:

    1. **Study the basics:** Study basic mathematical concepts such as distributive laws (the law governing how things move around) before starting any new project. This will help develop your understanding quickly while still being able to apply it later if needed. 
      2. **Develop strong teamwork relationships** : Develop strong teams where everyone works together well so they don't have too many conflicts between them. Make sure each person has their own strengths/skills which helps keep everything running smoothly